In [1]:
from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
def load_csv(filename):
    file = open(filename, "r", encoding="utf-8")
    experiments = []
    headers = []
    reader = csv.reader(file)
    for i, lines in enumerate(reader):
        if i == 0:
            #headers = [e for e in lines[0].split(";")]
            h = lines[0].split(";")
            headers.append(h)
        else:
            tmp = lines[0].split(";")
            for j in range(0,len(tmp)):
                if (j != 0 and j != 1879):
                    tmp[j] = float(tmp[j])
                else:
                    continue
            #tmp = [float(tmp[j]) for j in range(0,len(tmp)) if (j!=0 and j!=1879)] #This removes two entries, dunno why. 
            experiments.append(tmp)
    file.close()
    return headers, experiments

def column_extraction(data_list,column_number): #columns start counting in 0
    col = []
    for i in range(0,len(experiments)):
        dato = experiments[i][column_number]
        col.append(dato)
    return col   

## Load both data sets for comparision

In [3]:
# Loading compound library
my_sheet = 'SPECS_drug_repurp_set' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = r'/home/jovyan/covid-data/SPECS_drug_repurp_set.xlsx'# change it to the name of your excel file
df = read_excel(file_name, sheet_name = my_sheet)
#print(df.head()) # shows headers with top 5 rows
library_compound_ids = df[["Compound ID"]].copy() #Extracting ID's form specs
library_compound_ids.rename(columns = {'Compound ID':'ID_library'}, inplace = True)

In [4]:
#Loading Maris' data
headers, experiments = load_csv('/home/jovyan/covid-data/dalia_MeanFeaturesPerWell_BatchA.csv')

#Column 1879 contains the compound ID
screening_compound_ids = pd.DataFrame(column_extraction(experiments,1879), columns=['ID_Screening'])#Extracting ID's form covid

In [5]:
print(f'The library has {len(library_compound_ids)} compounds')
print(f'The experiment has {len(screening_compound_ids)} compounds')

The library has 5280 compounds
The experiment has 6065 compounds


### Analysing SPECS compound repetitions

In [6]:
# Removing compunds with ID "Problem with structure"
problem = library_compound_ids.loc[library_compound_ids['ID_library']=='Problem with structure']
problem

,ID_library
3471,Problem with structure
5185,Problem with structure


In [7]:
#Check that we did not miss information
library_compound_ids_2 = library_compound_ids.copy().drop([3471, 5185])
print(f'SPECS has now {len(library_compound_ids_2)} compounds')

SPECS has now 5278 compounds


In [8]:
# Finding repetitions
library_compound_ids_2 = library_compound_ids_2.copy().groupby('ID_library').size().sort_values(ascending=False).to_frame() #Count the elements in each group and sort
library_compound_ids_2.rename(columns = {0:'Count'}, inplace = True)

# Showing non-unique compounds
unique_id_lib_count = library_compound_ids_2.copy().reset_index()
a = unique_id_lib_count[unique_id_lib_count['Count'] >2]
print(f'Printing repeated compounds {a}')
repeated_compounds = unique_id_lib_count['ID_library'][unique_id_lib_count['Count'] >1].tolist() #Storing only repeated compounds


repeated_compounds.remove('CBK277930') # <------------------OUTLIER COMPOUND

Printing repeated compounds   ID_library  Count
0  CBK011558      3
1  CBK011603      3
2  CBK042067      3
3  CBK041250      3
4  CBK000879      3
5  CBK011551      3
6  CBK041748      3


## Create the data sets to use 

### SPECS data set

In [10]:
# Loading data
specs = df.copy().drop([3471, 5185])
print(f"There are {len(specs)} rows in SPECS")

# Remove outlier
print('Removing outlier...')
specs = specs.drop(specs.loc[specs['Compound ID']=='CBK277930'].index.tolist())
print(f"There are {len(specs)} rows in SPECS")

# Remove duplicates but keeping the fisrts appereance 
print('Removing duplicates...')
specs = specs.drop_duplicates(subset = 'Compound ID', keep = 'first')
print(f"There are {len(specs)} rows in SPECS")
specs.reset_index(drop=True, inplace=True)
specs.tail()

Removing outlier...
There are 5276 rows in SPECS
Removing duplicates...
There are 5187 rows in SPECS


,Library,Compound ID,Batch nr,CUSTOMER_ID,MOLFORMULA,MOLWEIGHT,NAME,VENDOR,ADD_INFO,SMILES,IUPAC_NAME,STEREOCHEMISTRY,pert_iname,clinical_phase,moa,target,disease_area,indication
5182,SPECS,CBK021021G,BJ1898358,M92352362,H2Mg3O12Si4,379.29,Talc,Selleck Chemicals,14807-96-6,O=[Mg].O=[Mg].O=[Mg].O=[Si]=O.O=[Si]=O.O=[Si]=...,dioxosilane;oxomagnesium;hydrate,NaN,talc,NaN,NaN,NaN,NaN,NaN
5183,SPECS,CBK278033,BJ1897713,M93148412,C28H31NO2.C4H6O6,563.65,Lasofoxifene (Tartrate),MedChemExpress,"CAS 190791-29-8, target Estrogen Receptor/ERR",OC1=CC=C2[C@@H](C3=CC=C(OCCN4CCCC4)C=C3)[C@@H]...,"(1R,2S)-2-phenyl-1-[4-(2-pyrrolidin-1-ylethoxy...","(1R,2S)-isomer",lasofoxifene,Launched,selective estrogen receptor modulator (SERM),ESR1|ESR2,orthopedics|oncology,osteoporosis|breast cancer
5184,SPECS,CBK290260,BJ1897904,M97302542,C2HCl2O2,150.93,dichloroacetate,Sigma-Aldrich,NaN,ClC(Cl)C(=O)[O-],"2,2-dichloroacetate",NaN,dichloroacetate,Phase 3,pyruvate dehydrogenase kinase inhibitor,NaN,NaN,NaN
5185,SPECS,CBK290471G,BJ1896172,M98279124,2C10H17NOS.2C10H17NOS.2HCl.H2O,244.78,cevimeline,Bio-Techne,NaN,C[C@@H](SC3)O[C@@]13CN2CCC1CC2.C[C@H](SC6)O[C@...,"cis-2-Methylspiro[1,3-oxathiolane-5,3'-quinucl...",NaN,cevimeline,Launched,acetylcholine receptor agonist,CHRM3,rheumatology,sjogren's syndrome
5186,SPECS,CBK011688P,BJ1894598,M98649031,C24H28ClN5O3,469.98,DIMENHYDRINATE,Microsource Discovery Systems,523-87-5,Cn1c2nc(Cl)[nH]c2c(=O)n(C)c1=O.CN(C)CCOC(c1ccc...,"2-benzhydryloxy-N,N-dimethyl-ethanamine;8-chlo...",NaN,dimenhydrinate,Launched,histamine receptor antagonist,HRH1,neurology/psychiatry|gastroenterology,motion sickness|nausea|vomiting


In [11]:
file_namn = r'/home/jovyan/covid-data/SPECS_ready.csv'
#df.to_csv(file_namn, sep=',', encoding='utf-8',index=False)

### Covid data set

In [52]:
# Loading dataframe
experiments_df = pd.DataFrame(experiments , columns=headers[0])
print(f"There are {len(experiments_df)} rows in Covid")

# Remove outlier
print('Removing outlier...\n')
covid_batchA = experiments_df.copy().drop(experiments_df.loc[experiments_df['CompoundID']=='CBK277930'].index.tolist())
print(f"There are {len(covid_batchA)} rows in Covid Batch A")

There are 6065 rows in Covid
Removing outlier...

There are 6063 rows in Covid Batch A


In [53]:
for compound in repeated_compounds:
    covid_batchA = covid_batchA.drop(covid_batchA.loc[covid_batchA['CompoundID'] == compound].index.tolist())

print(f"Removing repetitions...\n There are {len(covid_batchA)} rows in Covid Batch A")

Removing repetitions...
 There are 5893 rows in Covid Batch A


In [54]:
counter = 0
median_repeated_compounds_idx = []
for compound in repeated_compounds:
    
    np.random.seed(17)
    
    aux = experiments_df['Intensity_MeanIntensity_illumMITO_cells'].loc[experiments_df['CompoundID'] == compound] # Accessing info from the repeated compounds found in SPECS
    aux.columns = ['0','1'] #Renaming columns bcs of problem with column replication
    aux.drop('0', axis=1, inplace=True)
    aux2 = aux.sort_values(by=['1'], ascending=True) # Sort to access the mean
    
    # Non-permanent solution
    if len(aux2) == 2:
        median_index = np.random.choice(aux.index.tolist())
        median = aux2['1'][median_index]
    else:
        median = aux2.median(axis=0).tolist()
        median_index = aux2.index[aux2['1'] == median[0]].tolist()
        median_index = median_index[0]
        
    row = experiments_df.iloc[[median_index]]
    median_repeated_compounds_idx.append(median_index)
    covid_batchA = covid_batchA.append(row)
    
    #print(median_index)
    print(compound)
    #print(row)
    print(f'There are {len(covid_batchA)} rows in Covid Batch A')
    
    counter += 1
    if counter == 5:
        break

CBK011558
There are 5894 rows in Covid Batch A
CBK011603
There are 5895 rows in Covid Batch A
CBK042067
There are 5896 rows in Covid Batch A
CBK041250
There are 5897 rows in Covid Batch A
CBK000879
There are 5898 rows in Covid Batch A
